In [4]:
import torch
import numpy as np
import normflows as nf

torch.manual_seed(42)
torch.no_grad()

import sys
import os
c_directory = os.getcwd()
sys.path.append(os.path.join(c_directory, 'BSCD'))
sys.path.append(os.path.join(c_directory, 'FCYeast'))

from matplotlib import pyplot as plt
import FCYeast_simulator

enable_cuda = True
CUDA_LAUNCH_BLOCKING=1
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

model_file = 'FCYeast/FCYeast_network.pt'

RuntimeError: CUDA error: out of memory
CUDA kernel errors might be asynchronously reported at some other API call, so the stacktrace below might be incorrect.
For debugging consider passing CUDA_LAUNCH_BLOCKING=1.
Compile with `TORCH_USE_CUDA_DSA` to enable device-side assertions.


In [ ]:
target = FCYeast_simulator.target()
context_size = 6

latent_size = 1
hidden_units = 128
hidden_layers = 5

flows = [nf.flows.AutoregressiveRationalQuadraticSpline(latent_size, hidden_layers, 64,num_context_channels=context_size,tail_bound=30),
         nf.flows.LULinearPermute(latent_size),
         nf.flows.AutoregressiveRationalQuadraticSpline(latent_size, hidden_layers, 128,num_context_channels=context_size,tail_bound=30),
         nf.flows.LULinearPermute(latent_size),
         nf.flows.AutoregressiveRationalQuadraticSpline(latent_size, hidden_layers, 256,num_context_channels=context_size,tail_bound=30),
         nf.flows.LULinearPermute(latent_size),      
         nf.flows.AutoregressiveRationalQuadraticSpline(latent_size, hidden_layers, 512,num_context_channels=context_size,tail_bound=30),
         nf.flows.LULinearPermute(latent_size)
         ]

# Set base distribution
q0 = nf.distributions.DiagGaussian(1)
model = nf.ConditionalNormalizingFlow(q0, flows, target)
model = model.to(device)

for param in model.parameters():
    param.requires_grad = False

In [ ]:
model.load_state_dict(torch.load(model_file))

<All keys matched successfully>

In [7]:
import pandas as pd

dils = [.12,.23,.33]

dfs = [pd.read_csv('clean_data/complete_d={}.csv'.format(d)) for d in dils]
x = [torch.tensor(df['FL1-A'].to_numpy().astype(np.float32)).reshape(-1,1).to(device) for df in dfs]

ind = torch.arange(0,x[0].size(0),3)
xs = [torch.log(xi[ind]) for xi in x]

NameError: name 'device' is not defined

In [8]:
vectorize_params = torch.ones(x.size(0),5).to(device)

def log_likelihood(data,params,model):
    global vectorize_params
    if data.size !=  vectorize_params.size(0):
        vectorize_params = torch.ones((data.size(0),6),device=device)

    return model.log_prob(data,params*vectorize_params)

def log_post(data,params,model,lprior):
    return log_likelihood(data,params,model).sum() + lprior(params)

NameError: name 'x' is not defined

In [9]:
def separate(x):
    return torch.tensor(((x[0],x[1],x[2],x[3],x[4],x[11]),
                         (x[0],x[1],x[5],x[6],x[7],x[11]),
                         (x[0],x[1],x[8],x[9],x[10],x[11])))

to_hours = torch.tensor(((0,0,-1,-1,1,0),
                         (0,0,-1,-1,1,0),
                         (0,0,-1,-1,1,0)))*torch.log(torch.tensor(dils)).reshape(-1,1)  

def transform_to_hour(x): #suppose that \betas (0 and 1) are in hours, and the other in the arbirary units. Turn them all in hours
    return separate(x) + to_hours


to_arbitrary = torch.tensor(((1,1, 0, 0,0,0),
                             (1,1, 0, 0,0,0),
                             (1,1, 0, 0,0,0)))*torch.log(torch.tensor(dils)).reshape(-1,1)
def transform_to_arbitrary(x):  #suppose that \betas (0 and 1) are in hours, and the other in the arbirary units. Turn them all to arbitrary
    return separate(x) + to_arbitrary
    

In [ ]:
#make new prior
#make 5k initials from prior

tensor([ 4.3506,  7.1825,  0.5280, -1.2753, -2.7155, -0.6066], device='cuda:0') tensor(-84466.8906, device='cuda:0')
tensor([ 4.9074,  8.1992, -1.1204,  1.8577, -2.4786,  1.3762], device='cuda:0') tensor(-73961.3438, device='cuda:0')
tensor([ 3.1971,  6.4192,  0.8387,  1.4192, -2.1383,  0.8505], device='cuda:0') tensor(-73543.0781, device='cuda:0')
tensor([ 4.5427,  8.7605, -0.5787, -0.7050, -2.4808, -1.0141], device='cuda:0') tensor(-67088.5000, device='cuda:0')
tensor([ 4.3439,  8.7600, -0.6073,  0.3023, -2.6360, -0.6370], device='cuda:0') tensor(-61231.1641, device='cuda:0')
tensor([ 4.4364,  7.4674, -1.0163, -0.2045, -2.1691,  0.3886], device='cuda:0') tensor(-59646.8555, device='cuda:0')
tensor([ 4.2277,  8.6798, -0.0830,  1.5087, -1.9863, -0.3298], device='cuda:0') tensor(-58543.7266, device='cuda:0')


In [ ]:
#make mcmc adaptative

tensor([[ 2.1203,  2.1203,  2.1203,  2.1203, -2.1203, -0.0000],
        [ 1.4697,  1.4697,  1.4697,  1.4697, -1.4697, -0.0000],
        [ 1.1087,  1.1087,  1.1087,  1.1087, -1.1087, -0.0000]])

In [ ]:
#make mcmc final

In [ ]:
separate_dils = torch.tensor(((-1,-1,-1,-1,1,0),
                              (-1,-1,-1,-1,1,0),
                              (-1,-1,-1,-1,1,0)))*torch.log(torch.tensor(dils)).reshape(-1,1)

In [ ]:
separate_dils

tensor([[ 2.1203,  2.1203,  2.1203,  2.1203, -2.1203, -0.0000],
        [ 1.4697,  1.4697,  1.4697,  1.4697, -1.4697, -0.0000],
        [ 1.1087,  1.1087,  1.1087,  1.1087, -1.1087, -0.0000]])

In [10]:
to_hours

tensor([[-0.0000, -0.0000,  2.1203,  2.1203, -2.1203, -0.0000],
        [-0.0000, -0.0000,  1.4697,  1.4697, -1.4697, -0.0000],
        [-0.0000, -0.0000,  1.1087,  1.1087, -1.1087, -0.0000]])

In [11]:
to_arbitrary

tensor([[2.1203, 2.1203, -0.0000, -0.0000, -0.0000, -0.0000],
        [1.4697, 1.4697, -0.0000, -0.0000, -0.0000, -0.0000],
        [1.1087, 1.1087, -0.0000, -0.0000, -0.0000, -0.0000]])

In [5]:
import torch